In [2]:
%gui qt6

# Qt6 stuff
from PyQt6.QtWidgets import QApplication
from PyQt6 import QtWidgets
from PyQt6 import QtGui
from PyQt6 import QtCore
from PyQt6.QtCore import pyqtSignal

# Plotting stuff
import pyqtgraph as pg
import matplotlib.pyplot as plt
import cmasher as cmr

# Science stuff
import numpy as np
import pandas as pd
from spectres import spectres

# Astropy stuff
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.units.quantity import Quantity
from astropy.io.ascii import read as ascii_read

# zHunter stuff
from zhunter import DIRS
from zhunter import io
from zhunter.misc import set_up_linked_vb
from zhunter.colors import get_gradient
from zhunter.spectroscopic_system import SpecSystemModel, SpecSystem

# General stuff
import logging
import sys
from pathlib import Path
from itertools import cycle

In [3]:
log = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s [%(name)s] %(message)s')
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING)

In [4]:
# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance

In [5]:
from zhunter.colors import COLORS
color_style = 'kraken9'
colors = COLORS[color_style]

In [24]:
# from pyqtgraph.Qt import QtCore
from collections import defaultdict

# Get key mappings from Qt namespace
qt_keys = (
    (getattr(QtCore.Qt.Key, attr), attr[4:])
    for attr in dir(QtCore.Qt.Key)
    if attr.startswith("Key_")
)
keys_mapping = defaultdict(lambda: "unknown", qt_keys)
 
    
# Get event mappings from Qt namespace
qt_events = (
    (getattr(QtCore.QEvent.Type, event).name, getattr(QtCore.QEvent.Type, event).value)
    for event in dir(QtCore.QEvent.Type)
    if not event.startswith("_")
)
events_mapping = defaultdict(lambda: "unknown", qt_events)
 

2023-03-16 17:48:43,155 DEBUG [__main__] keyPressEvent! Key: E, Mouse position: [302,242]


Traceback (most recent call last):
  File "/var/folders/r1/qchx86rn3tx643ytrjxf6pmm0000gn/T/ipykernel_17050/4241388917.py", line 122, in keyPressEvent
    for ax in self.axes:
AttributeError: 'MainPlotWidget' object has no attribute 'axes'


2023-03-16 17:48:44,360 DEBUG [__main__] keyPressEvent! Key: E, Mouse position: [343,145]


Traceback (most recent call last):
  File "/var/folders/r1/qchx86rn3tx643ytrjxf6pmm0000gn/T/ipykernel_17050/4241388917.py", line 122, in keyPressEvent
    for ax in self.axes:
AttributeError: 'MainPlotWidget' object has no attribute 'axes'


2023-03-16 17:48:44,787 DEBUG [__main__] keyPressEvent! Key: Q, Mouse position: [282,137]


Traceback (most recent call last):
  File "/var/folders/r1/qchx86rn3tx643ytrjxf6pmm0000gn/T/ipykernel_17050/4241388917.py", line 122, in keyPressEvent
    for ax in self.axes:
AttributeError: 'MainPlotWidget' object has no attribute 'axes'


2023-03-16 17:48:45,114 DEBUG [__main__] keyPressEvent! Key: E, Mouse position: [190,382]


Traceback (most recent call last):
  File "/var/folders/r1/qchx86rn3tx643ytrjxf6pmm0000gn/T/ipykernel_17050/4241388917.py", line 122, in keyPressEvent
    for ax in self.axes:
AttributeError: 'MainPlotWidget' object has no attribute 'axes'


In [33]:
class MainPlotWidget(pg.GraphicsLayoutWidget):
    """
    Main Plot class which subclasses `GraphicsLayoutWidget` and
    installs handling of key presses.
    """

    def __init__(self, *args, **kwds):
        super().__init__(*args, **kwds)
        self.mousePoint = QtCore.QPointF()
        # To suppress qt.dispatch warning
        self.viewport().setAttribute(
            QtCore.Qt.WidgetAttribute.WA_AcceptTouchEvents,
            False,
        )
        # Connect scene mouse moved to own mouse point
        # to keep track of mouse position at all times
        # (used for knowing mouse position during keyPressEvents)
        self.scene().sigMouseMoved.connect(self.update_mouse_pos)

    def update_mouse_pos(self, pos):
        self.mousePoint = self.mapFromScene(pos)

    def eventFilter(self, widget, event):
        if event.isExit():
            print(event)
        return QtWidgets.QWidget.eventFilter(self, widget, event)


In [34]:
win = MainPlotWidget(show=True)

Traceback (most recent call last):
  File "/var/folders/r1/qchx86rn3tx643ytrjxf6pmm0000gn/T/ipykernel_17050/826727759.py", line 24, in eventFilter
    if event.isExit():
AttributeError: 'QEvent' object has no attribute 'isExit'
Traceback (most recent call last):
  File "/var/folders/r1/qchx86rn3tx643ytrjxf6pmm0000gn/T/ipykernel_17050/826727759.py", line 24, in eventFilter
    if event.isExit():
AttributeError: 'QEvent' object has no attribute 'isExit'


In [30]:
win.installEventFilter(win)

In [8]:
def ev_catcher(*args):
    print("Got signal !")
    print(args)

In [35]:
ev = getattr(QtCore.QEvent.Type, 'KeyPress')
ev

<Type.KeyPress: 6>

In [36]:
events_mapping

defaultdict(<function __main__.<lambda>()>,
            {'ActionAdded': 114,
             'ActionChanged': 113,
             'ActionRemoved': 115,
             'ActivationChange': 99,
             'ApplicationActivate': 121,
             'ApplicationDeactivate': 122,
             'ApplicationFontChange': 36,
             'ApplicationLayoutDirectionChange': 37,
             'ApplicationPaletteChange': 38,
             'ApplicationStateChange': 214,
             'ApplicationWindowIconChange': 35,
             'ChildAdded': 68,
             'ChildPolished': 69,
             'ChildRemoved': 71,
             'Clipboard': 40,
             'Close': 19,
             'CloseSoftwareInputPanel': 200,
             'ContentsRectChange': 178,
             'ContextMenu': 82,
             'CursorChange': 183,
             'DeferredDelete': 52,
             'DragEnter': 60,
             'DragLeave': 62,
             'DragMove': 61,
             'Drop': 63,
             'DynamicPropertyChange': 170,
   

In [37]:
sb = QtWidgets.QStatusBar()

In [ ]:
sb.clearMessage

In [31]:
scene = win.scene()

In [33]:
scene.sigMouseMoved.connect(ev_catcher)

Got signal !
(PyQt6.QtCore.QPointF(289.0, 233.0),)
Got signal !
(PyQt6.QtCore.QPointF(289.0, 233.0),)
Got signal !
(PyQt6.QtCore.QPointF(289.0, 230.0),)
Got signal !
(PyQt6.QtCore.QPointF(292.0, 225.0),)
Got signal !
(PyQt6.QtCore.QPointF(292.0, 223.0),)
Got signal !
(PyQt6.QtCore.QPointF(293.0, 220.0),)
Got signal !
(PyQt6.QtCore.QPointF(294.0, 216.0),)
Got signal !
(PyQt6.QtCore.QPointF(294.0, 214.0),)
Got signal !
(PyQt6.QtCore.QPointF(294.0, 212.0),)
Got signal !
(PyQt6.QtCore.QPointF(294.0, 212.0),)
Got signal !
(PyQt6.QtCore.QPointF(294.0, 211.0),)
Got signal !
(PyQt6.QtCore.QPointF(294.0, 211.0),)
Got signal !
(PyQt6.QtCore.QPointF(292.0, 210.0),)
Got signal !
(PyQt6.QtCore.QPointF(289.0, 208.0),)
Got signal !
(PyQt6.QtCore.QPointF(286.0, 205.0),)
Got signal !
(PyQt6.QtCore.QPointF(282.0, 204.0),)
Got signal !
(PyQt6.QtCore.QPointF(276.0, 201.0),)
Got signal !
(PyQt6.QtCore.QPointF(271.0, 198.0),)
Got signal !
(PyQt6.QtCore.QPointF(262.0, 196.0),)
Got signal !
(PyQt6.QtCore.QPoi

In [46]:
scene.sigMouseClicked.connect(ev_catcher)

Got signal !
(<MouseClickEvent (117.312,69) button=MouseButton.LeftButton>,)
Got signal !
(<MouseClickEvent (126.312,115) button=MouseButton.RightButton>,)
Got signal !
(<MouseClickEvent (130.526,122) button=MouseButton.LeftButton>,)
Got signal !
(<MouseClickEvent (193.312,87) button=MouseButton.LeftButton>,)
Got signal !
(<MouseClickEvent (13,18) button=MouseButton.LeftButton>,)
Got signal !
(<MouseClickEvent (619,466) button=MouseButton.LeftButton>,)
